In [ ]:
import os
from database_tools.tools import BuildDatabase

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

config = dict(
    low=0.5,                    # low bandpass frequency
    high=8.0,                   # high bandpass frequency
    sim=0.6,                    # similarity threshold
    df=0.2,                     # one sided frequency delta for SNR calculation
    snr_t=2.0,                  # SNR threshold
    hr_diff=1/6,                # maximum heart rate difference between ppg, abp
    f0_low=0.667,               # minimum valid HR in Hz
    f0_high=3.0,                # maximum valid HR in Hz
    abp_min_bounds=[40, 100],   # upper and lower threshold for DBP
    abp_max_bounds=[70, 190],   # upper and lower threshold for SBP
)

worker = BuildDatabase(
    output_dir='data-2022-10-28/',
    config=config,
    win_len=256,
    fs=125,
    samples_per_file=2500,
    samples_per_patient=500,
    max_samples=300000,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)

worker.run()

In [ ]:
import os
import pandas as pd
from database_tools.tools import DataEvaluator

pd.options.display.max_rows = 2500

df = pd.read_csv('../data-2022-10-28/mimic3_stats.csv')
worker = DataEvaluator(stats=df)

figs = worker.run()

In [5]:
import os
import time
import glob
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

ppg = []
abp = []
with open('../Enceladus/data-2022-10-28/mimic3/lines/mimic3_0000000.jsonlines', 'r') as f:
    for sample in f:
        sample = json.loads(sample)
        ppg.append(sample['ppg'])
        abp.append(sample['abp'])

ppg = np.array(ppg)
abp = np.array(abp)

fig = go.FigureWidget()
fig.add_scatter()
fig.add_scatter()
fig.update_layout(
    template='plotly_dark',
)

FigureWidget({
    'data': [{'type': 'scatter', 'uid': 'a9b6b555-27e3-494f-9282-f2a346c1680b'},
             {…

In [6]:
from sklearn.metrics import mean_absolute_error

fig.data[0].y = ppg[0, :]
fig.data[1].y = abp[0, :]